## Chap06-01. MNIST
### MNIST(손글씨 숫자 인식) 문제를 신경망으로 풀어보기

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 1. 데이터 로드
#  - Tensorflow에 기본 내장된 mnist 모듈을 이용하여 데이터를 로드
#  - 지정한 폴더에 MNIST 데이터가 없는 경우 자동으로 데이터를 다운로드함
#  - one-hot 옵션은 Label을 one-hot vector로 만들어 줌
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 2. 신경망 모델 구성

# 입력값(input value)의 차원은 [배치크기, 특성값]으로 되어 있음
# MNIST 이미지는 28x28 (=784) 픽셀로 이루어져 있고, 이를 784의 특성값으로 정함

# 1) Input(X), Label(Y) 설정
#   - Input(X): 784(28x28)을 Input value로 설정
#   - Output(Y): [0~9] 즉, 10개의 class를 가짐
#   - MiniBatch: Batch size는 None으로 설정
X = tf.placeholder(tf.float32, [None, 784]) 
Y = tf.placeholder(tf.float32, [None, 10])


# 2) 신경망 레이어(layer)구성
# Input[Batch,784] -> Hidden(1)[256] -> Hidden(2)[256] -> Output[Batch, 10]
# 1st layer
W1 = tf.get_variable(name='W1', shape=[784, 256], 
                     initializer=tf.contrib.layers.xavier_initializer())
L1 = tf.nn.relu(tf.matmul(X, W1))

# 2nd layer
W2 = tf.get_variable(name='W2', shape=[256, 256], 
                     initializer=tf.contrib.layers.xavier_initializer())
L2 = tf.nn.relu(tf.matmul(L1, W2))

# output layer
W3 = tf.get_variable(name='W3', shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
model = tf.matmul(L2, W3)

In [4]:
# 3) 손실함수(Loss function) 설정
cost = tf.losses.softmax_cross_entropy(logits=model, onehot_labels=Y)
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [5]:
# 3. 신경망 모델 학습

# 1) 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 2) Batch size 설정
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

# 3) 최적화 진행
for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        # Tensorflow의 Mnist 모델의 next_batch 함수를 이용해
        # batch size 만큼 학습할 데이터를 가져옴
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch: {:04d} Avg. cost = {:.3f}'.format((epoch + 1), (total_cost / total_batch)))
    
print("최적화 완료!")

Epoch: 0001 Avg. cost = 0.255
Epoch: 0002 Avg. cost = 0.099
Epoch: 0003 Avg. cost = 0.066
Epoch: 0004 Avg. cost = 0.046
Epoch: 0005 Avg. cost = 0.034
Epoch: 0006 Avg. cost = 0.027
Epoch: 0007 Avg. cost = 0.021
Epoch: 0008 Avg. cost = 0.018
Epoch: 0009 Avg. cost = 0.016
Epoch: 0010 Avg. cost = 0.016
Epoch: 0011 Avg. cost = 0.014
Epoch: 0012 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.010
Epoch: 0014 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.007
최적화 완료!


In [9]:
# 4. 결과 확인
# Model로 예측한 값과 실제 레이블인 Y의 값을 비교
# tf.argmax 함수를 이용해 예측한 값에서 가장 큰 값의 class로 분류
# 예) [0.1 0 0 0.7 0 0.2 0 0 0 0] -> 3
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도:", sess.run(accuracy, feed_dict={X: mnist.test.images,
                                                Y: mnist.test.labels}))

정확도: 0.9803
